In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os.path
import glob
from urllib.request import urlretrieve
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import os
import sys
import pathlib

# Imported because Roberto also did it.
# from torch_geometric.data import Data

# Load the data

In [104]:
path = pathlib.Path().resolve()
proj_dir = str(path.parent)
path_raw = proj_dir + "\data/raw_datasets"
path_aug = proj_dir + "\data/augmented_data"
print(path_raw)
print(path_aug)

C:\Users\sagi8\Documents\GitHub\DS-AI\data\processed_data\normalized_test_data\test3\data/raw_datasets
C:\Users\sagi8\Documents\GitHub\DS-AI\data\processed_data\normalized_test_data\test3\data/augmented_data


#    Reading me!!!

Load the augmented validation and training data. (from Patrick)\
Load the original testing data (Raw dataset). 

In [35]:
loading_variable = ['/DEM',"/WD","/VX","/VY"]
test = ['/test1','/test2','/test3']

dem_tra_val = []
dem_tst1 = []
dem_tst2 = []
dem_tst3 = []

wd_tra_val = []
wd_tst1 = []
wd_tst2 = []
wd_tst3 = []

vx_tra_val = []
vx_tst1 = []
vx_tst2 = []
vx_tst3 = []

vy_tra_val = []
vy_tst1 = []
vy_tst2 = []
vy_tst3 = []

var_list = [[dem_tra_val,dem_tst1,dem_tst2,dem_tst3],
            [wd_tra_val,wd_tst1,wd_tst2,wd_tst3],
            [vx_tra_val,vx_tst1,vx_tst2,vx_tst3],
            [vy_tra_val,vy_tst1,vy_tst2,vy_tst3]]

for i in range(len(loading_variable)):
    tv = glob.glob(path_aug + loading_variable[i] + '\*.txt')
    t1 = glob.glob(path_raw + test[0] + loading_variable[i] + '\*.txt')
    t2 = glob.glob(path_raw + test[1] + loading_variable[i] + '\*.txt')
    t3 = glob.glob(path_raw + test[2] + loading_variable[i] + '\*.txt')

    tem_var = var_list[i]

    for j in range(len(tv)):
        tem_var[0].append(np.loadtxt(tv[j]))

    for j in range(len(t1)):
        tem_var[1].append(np.loadtxt(t1[j]))

    for j in range(len(t2)):
        tem_var[2].append(np.loadtxt(t2[j]))

    for j in range(len(t3)):
        tem_var[3].append(np.loadtxt(t3[j]))

In [36]:
print(np.shape(dem_tra_val))
print(np.shape(dem_tst1))
print(np.shape(dem_tst2))
print(np.shape(dem_tst3))

(400, 4096, 3)
(20, 4096, 3)
(21, 4096, 3)
(10, 16384, 3)


# Split the train and validation dataset 

In [37]:
# Split the dataset (in total 80) into following portion: training(70%) and validation(30%)
# We already have the testing dataset
# DEM
dem_tra, dem_val, wd_tra, wd_val, vx_tra, vx_val, vy_tra, vy_val, idem_tra, idem_val = train_test_split(
    dem_tra_val, wd_tra_val, vx_tra_val, vy_tra_val, np.arange(len(dem_tra_val)), train_size=0.7, shuffle=True, random_state=42)


print(f"len(vx_tra): {len(dem_tra)}")
print(f"len(vx_val): {len(dem_val)}")
print(f"len(vx_tst1): {len(dem_tst1)}")
print(f"len(vx_tst2): {len(dem_tst2)}")
print(f"len(vx_tst3): {len(dem_tst3)}")

# Note that the form of the all the training, testing and validating dataset are "a list containing multiple arrays".

len(vx_tra): 280
len(vx_val): 120
len(vx_tst1): 20
len(vx_tst2): 21
len(vx_tst3): 10


In [38]:
# Here use the Min-max normalization to normalize the water depth over the entire simulation sequence (0-130)
def scale_sequences(X,scaler=None,scaler_type='minmax'):
    """
    Uses a minmax scaler to transform sequences. The scaler is created if no scaler is passed as argument.
    Adapted from exercise notebook on drinking water demand.

    The input parameter X is a two-dimensional array.
    """

    Xshape=X.shape
    if scaler:
        X = scaler.transform(X.reshape(-1,1)).reshape(Xshape)
        return X
    else:
        if scaler_type == 'standard':
            scaler = StandardScaler()
        elif scaler_type == 'minmax':
            scaler = MinMaxScaler()
        else:
            raise Exception("Type of scikit-learn scaler not supported. Choose 'standard' or 'minmax.")
        X = scaler.fit_transform(X.reshape(-1,1)).reshape(Xshape)
        return X, scaler

def denormalize(image_tensor, mean, std):
    # Denormalize the image
    denorm_img = image_tensor * std[:, None, None] + mean[:, None, None]
    # Clip values to be between 0 and 1
    denorm_img = denorm_img.clip(0, 1)
    return denorm_img


In [40]:
tra_arr = [dem_tra[0], wd_tra[0], vx_tra[0], vy_tra[0]]
val_arr = [dem_val[0], wd_val[0], vx_val[0], vy_val[0]]
tst1_arr = [dem_tst1[0], wd_tst1[0], vx_tst1[0], vy_tst1[0]]
tst2_arr = [dem_tst2[0], wd_tst2[0], vx_tst2[0], vy_tst2[0]]
tst3_arr = [dem_tst3[0], wd_tst3[0], vx_tst3[0], vy_tst3[0]]

splitlis = [[dem_tra, dem_val, dem_tst1, dem_tst2, dem_tst3],
             [wd_tra, wd_val, wd_tst1, wd_tst2, wd_tst3],
             [vx_tra, vx_val, vx_tst1, vx_tst2, vx_tst3],
             [vy_tra, vy_val, vy_tst1, vy_tst2, vy_tst3]]

scalelist = [[dem_tra_scale, dem_val_scale, dem_tst1_scale, dem_tst2_scale, dem_tst3_scale],
             [wd_tra_scale, wd_val_scale, wd_tst1_scale, wd_tst2_scale, wd_tst3_scale],
             [vx_tra_scale, vx_val_scale, vx_tst1_scale, vx_tst2_scale, vx_tst3_scale],
             [vy_tra_scale, vy_val_scale, vy_tst1_scale, vy_tst2_scale, vy_tst3_scale]]


datalist_tra = ["dem_tra_norm","wd_tra_norm", 'vx_tra_norm', 'vy_tra_norm']

datalist_val = ["dem_val_norm","wd_val_norm", 'vx_val_norm', 'vy_val_norm']

datalist_tst1 = ['dem_tst1_norm','wd_tst1_norm', 'vx_tst1_norm', 'vy_tst1_norm']

datalist_tst2 = ['dem_tst2_norm','wd_tst2_norm', 'vx_tst2_norm', 'vy_tst2_norm']

datalist_tst3 = ['dem_tst3_norm', 'wd_tst3_norm', 'vx_tst3_norm', 'vy_tst3_norm']

# Direct to Folders for saving
save_path_tra = proj_dir + "\data\processed_data/normalized_training_data"
save_path_val = proj_dir + "\data\processed_data/normalized_validation_data"
save_path_tst1 = proj_dir + "\data\processed_data/normalized_test_data/test1"
save_path_tst2 = proj_dir + "\data\processed_data/normalized_test_data/test2"
save_path_tst3 = proj_dir + "\data\processed_data/normalized_test_data/test3"

name = ["DEM","WD", "VX", "VY"]
fmt = '%1.4f'

In [109]:
display(len(splitlis[0][0]))

280

In [54]:
for i in  range(len(tra_arr)):
    tra_con = tra_arr[i]
    val_con = val_arr[i]
    tst1_con = tst1_arr[i]
    tst2_con = tst2_arr[i]
    tst3_con = tst3_arr[i]

    # training
    for j in range(1,len(dem_tra)):
        tra_con = np.concatenate((tra_con, splitlis[i][0][j]),axis=0)

    # validation
    for j in range(1,len(dem_val)):
        val_con = np.concatenate((val_con, splitlis[i][1][j]),axis=0)

    # test 1
    for j in range(1,len(dem_tst1)):
        tst1_con = np.concatenate((tst1_con, splitlis[i][2][j]),axis=0)

    #test 2
    for j in range(1,len(dem_tst2)):
        tst2_con = np.concatenate((tst2_con, splitlis[i][3][j]),axis=0)

    #test 3
    for j in range(1,len(dem_tst3)):
        tst3_con = np.concatenate((tst3_con, splitlis[i][4][j]),axis=0)

    tra_scale, scaler = scale_sequences(tra_con ,scaler=None,scaler_type='minmax')
    val_scale, scaler = scale_sequences(val_con ,scaler=None,scaler_type='minmax')
    tst1_scale, scaler = scale_sequences(tst1_con ,scaler=None,scaler_type='minmax')
    tst2_scale, scaler = scale_sequences(tst2_con ,scaler=None,scaler_type='minmax')
    tst3_scale, scaler = scale_sequences(tst3_con ,scaler=None,scaler_type='minmax')

    tra_norm = np.vsplit(tra_scale,len(dem_tra))
    val_norm = np.vsplit(val_scale,len(dem_val))
    tst1_norm = np.vsplit(tst1_scale,len(dem_tst1))
    tst2_norm = np.vsplit(tst2_scale,len(dem_tst2))
    tst3_norm = np.vsplit(tst3_scale,len(dem_tst3))


    os.chdir(f'{save_path_tra}/{name[i]}')
    for j in range(len(tra_norm)):
        np.savetxt(f'{datalist_tra[i]}{j}',tra_norm[j], fmt = fmt)


    os.chdir(f'{save_path_val}/{name[i]}')
    for j in range(len(val_norm)):
        np.savetxt(f'{datalist_val[i]}{j}',val_norm[j], fmt = fmt)


    os.chdir(f'{save_path_tst1}/{name[i]}')
    for j in range(len(tst1_norm)):
        np.savetxt(f'{datalist_tst1[i]}{j}',tst1_norm[j], fmt = fmt)


    os.chdir(f'{save_path_tst2}/{name[i]}')
    for j in range(len(tst2_norm)):
        np.savetxt(f'{datalist_tst2[i]}{j}',tst2_norm[j], fmt = fmt)


    os.chdir(f'{save_path_tst3}/{name[i]}')
    for j in range(len(tst3_norm)):
        np.savetxt(f'{datalist_tst3[i]}{j}',tst3_norm[j], fmt = fmt)
